# 1.サンプル学習データのダウンロード

In [ ]:
!wget https://hiouchiystorage.blob.core.windows.net/share/data.zip

In [ ]:
!mkdir train_data

In [ ]:
!mv data.zip train_data/

In [ ]:
!unzip train_data/data.zip -d train_data/

# 2.モデルの学習（MobileNet v1を転移学習）

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, CSVLogger
from tensorflow.keras import optimizers, models
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import applications
from tensorflow.keras import backend as K
import tensorflow as tf
import os

In [ ]:
test_path = 'train_data/test/'
train_path = 'train_data/train/'
val_path = 'train_data/val/'
WIDTH=224
HEIGHT=224
BATCH_SIZE=64

#Train DataSet Generator with Augmentation
print("\nTraining Data Set")
train_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
train_flow = train_generator.flow_from_directory(
    train_path,
    target_size=(HEIGHT, WIDTH),
    batch_size = BATCH_SIZE
)

#Validation DataSet Generator with Augmentation
print("\nValidation Data Set")
val_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
val_flow = val_generator.flow_from_directory(
    val_path,
    target_size=(HEIGHT, WIDTH),
    batch_size = BATCH_SIZE
)

#Test DataSet Generator with Augmentation
print("\nTest Data Set")
test_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
test_flow = test_generator.flow_from_directory(
    test_path,
    target_size=(HEIGHT, WIDTH),
    batch_size = BATCH_SIZE
)


# Initialize MobileNet with transfer learning
base_model = applications.MobileNet(weights='imagenet', 
                                include_top=False, 
                                input_shape=(WIDTH, HEIGHT,3))

# add a global spatial average pooling layer
x = base_model.output

x = GlobalAveragePooling2D()(x)
# and a dense layer
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(train_flow.class_indices), activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional MobileNet layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=optimizers.Adam(lr=0.001), metrics=['accuracy', 'top_k_categorical_accuracy'], loss='categorical_crossentropy')
model.summary()


import math
top_layers_file_path="mobilenet.hdf5"

checkpoint = ModelCheckpoint(top_layers_file_path, monitor='loss', verbose=1, save_best_only=True, mode='min')
tb = TensorBoard(log_dir='./logs', batch_size=val_flow.batch_size, write_graph=True, update_freq='batch')
early = EarlyStopping(monitor="loss", mode="min", patience=5)
csv_logger = CSVLogger('./logs/mn-log.csv', append=True)

history = model.fit_generator(train_flow, 
                              epochs=1, 
                              verbose=1,
                              validation_data=val_flow,
                              validation_steps=math.ceil(val_flow.samples/val_flow.batch_size),
                              steps_per_epoch=math.ceil(train_flow.samples/train_flow.batch_size),
                              callbacks=[checkpoint, early, tb, csv_logger])


model.load_weights(top_layers_file_path)
loss, acc, top_5 = model.evaluate_generator(
    test_flow,
    verbose = True,
    steps=math.ceil(test_flow.samples/test_flow.batch_size))
print("Loss: ", loss)
print("Acc: ", acc)
print("Top 5: ", top_5)


label = [k for k,v in train_flow.class_indices.items()]
with open('labels.txt', 'w+') as file:
    file.write("\n".join(label))
 
tf.saved_model.save(model, 'mobilenet')

# 3.モデルの推論

In [ ]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import cv2
import time
import glob
import random
import pandas as pd
from PIL import Image
import PIL
import io
import argparse
import sys
from openvino.inference_engine import IECore

In [ ]:
loaded = tf.saved_model.load('saved_model')
#print(list(loaded.signatures.keys()))  # ["serving_default"]

infer = loaded.signatures["serving_default"]
output_node_name = next(iter(infer.structured_outputs.keys()))

#Read in Labels
arg_labels="labels.txt"
label_file = open(arg_labels, "r")
labels = label_file.read().split('\n')

file_list = glob.glob("train_data/test/*/*")
for i in range(50):
    img_path = random.choice(file_list)
    img_cat = os.path.split(os.path.dirname(img_path))[1]
    
    start1 = time.time()
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=[224, 224])
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = tf.keras.applications.mobilenet.preprocess_input(x[tf.newaxis,...])

    start2 = time.time() #ここ追加
    labeling = infer(tf.constant(x))[output_node_name]
    infer_time = time.time() - start2
    total_time = time.time() - start1
    print("Filename:{}, Prediction:{}, ProcTime:{}, InferTime:{}".format(img_path, labels[np.argsort(labeling)[0,::-1][0]], int(total_time*1000), int(infer_time*1000)))

TensorFlowでの作業は以上となります。

---
ここからOpenVINO↓
# 4.OpenVINOでFP32モデルをCPUに最適化（IRに変換）

ここからはIntel® OpenVINO™ Toolkitを用いた量子化方法をご紹介します。

といってもまずは、元のTensorFlowのモデル（FP32）をOpenVINOのIR（Intermidiate Repretation）形式に変換するところから実施しましょう。

In [ ]:
!python3 /opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py --saved_model_dir ./saved_model --input_shape=[1,224,224,3] --data_type FP16

IR(xml+bin)が生成されていることを確認します。

# 5.最適化済みのモデルをOpenVINOの推論エンジン（Inference Engine）上で実行

In [ ]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import cv2
import time
import glob
import random
import pandas as pd
from PIL import Image
import PIL
import io
import argparse
import sys
from openvino.inference_engine import IECore

In [ ]:
class Model(object):

    def __init__(self):
        self.labels = []
        labels_filename = "labels.txt"
        #Read in Labels
        arg_labels="labels.txt"
        label_file = open(arg_labels, "r")
        self.labels = label_file.read().split('\n')

    def predict(self, imageFile):
        raise NotImplementedError
    
    def convert_to_opencv(self, image):
        # RGB -> BGR conversion is performed as well.
        image = image.convert('RGB')
        r,g,b = np.array(image).T
        opencv_image = np.array([b,g,r]).transpose()
        return opencv_image

    def crop_center(self, img,cropx,cropy):
        h, w = img.shape[:2]
        startx = w//2-(cropx//2)
        starty = h//2-(cropy//2)
        return img[starty:starty+cropy, startx:startx+cropx]

    def resize_down_to_1600_max_dim(self, image):
        h, w = image.shape[:2]
        if (h < 1600 and w < 1600):
            return image

        new_size = (1600 * w // h, 1600) if (h > w) else (1600, 1600 * h // w)
        return cv2.resize(image, new_size, interpolation = cv2.INTER_LINEAR)

    def resize_to_256_square(self, image):
        h, w = image.shape[:2]
        return cv2.resize(image, (256, 256), interpolation = cv2.INTER_LINEAR)

    def update_orientation(self, image):
        exif_orientation_tag = 0x0112
        if hasattr(image, '_getexif'):
            exif = image._getexif()
            if (exif != None and exif_orientation_tag in exif):
                orientation = exif.get(exif_orientation_tag, 1)
                # orientation is 1 based, shift to zero based and flip/transpose based on 0-based values
                orientation -= 1
                if orientation >= 4:
                    image = image.transpose(Image.TRANSPOSE)
                if orientation == 2 or orientation == 3 or orientation == 6 or orientation == 7:
                    image = image.transpose(Image.FLIP_TOP_BOTTOM)
                if orientation == 1 or orientation == 2 or orientation == 5 or orientation == 6:
                    image = image.transpose(Image.FLIP_LEFT_RIGHT)
        return image

class OpenVINOModel(Model):

    def __init__(self, target_device, modelFilePath):
        super(OpenVINOModel, self).__init__()

        # These are set to the default names from exported models, update as needed.
        model_xml = modelFilePath
        model_bin = modelFilePath.replace('.xml', '.bin')

        # Plugin initialization for specified device and load extensions library if specified
        # Set the desired device name as 'device' parameter. This sample support these 3 names: CPU, GPU, MYRIAD
        ie = IECore()

        # Read IR
        self.net = ie.read_network(model=model_xml, weights=model_bin)

        self.input_blob = next(iter(self.net.inputs))
        self.out_blob = next(iter(self.net.outputs))
        self.net.batch_size = 1

        # Loading model to the plugin
        self.exec_net = ie.load_network(network=self.net, device_name='CPU', num_requests=1)

    def predict(self, imageFile):
        start1 = time.time() #ここ追加

        # Load from a file
        image = Image.open(imageFile)

        # Update orientation based on EXIF tags, if the file has orientation info.
        image = super().update_orientation(image)

        # Convert to OpenCV format
        image = super().convert_to_opencv(image)

        # If the image has either w or h greater than 1600 we resize it down respecting
        # aspect ratio such that the largest dimension is 1600
        image = super().resize_down_to_1600_max_dim(image)

        # We next get the largest center square
        h, w = image.shape[:2]
        min_dim = min(w,h)
        max_square_image = super().crop_center(image, min_dim, min_dim)

        # Resize that square down to 256x256
        augmented_image = super().resize_to_256_square(max_square_image)

        # Get the input size of the model
        n, c, h, w = self.net.inputs[self.input_blob].shape

        # Crop the center for the specified network_input_Size
        augmented_image = super().crop_center(augmented_image, w, h)
        frame = augmented_image

        #
        augmented_image = augmented_image.transpose((2, 0, 1))

        images = np.ndarray(shape=(n, c, h, w))
        images[0] = augmented_image

        start2 = time.time() #ここ追加
        predictions = self.exec_net.infer(inputs={self.input_blob: images})
        infer_time = time.time() - start2

        # Print the highest probability label
        predictions = predictions[self.out_blob]
        highest_probability_index = predictions[0].argsort()[-1:][::-1][0]

        total_time = time.time() - start1

        return total_time, infer_time, self.labels[highest_probability_index], frame  #ここ追加


def run_inference(modelFile, model_type="tf", target_device='CPU', total=500):
    if model_type == 'tf':
        model = TFModel(modelFile)
    elif model_type == 'tf_int8':
        model = TFModel(modelFile)
    else:
        if target_device == 'GPU':
            model = OpenVINOModel('GPU', modelFile)
        elif target_device == 'MYRIAD':
            model = OpenVINOModel('MYRIAD', modelFile)
        else:
            model = OpenVINOModel('CPU', modelFile)

    total_infer_spent_time = 0
    total_spent_time = 0
    list_df = pd.DataFrame( columns=['正解ラベル','予測ラベル','全処理時間(msec)','推論時間(msec)'] )

    match = 0
    #file_list = glob.glob(os.path.join(dataset_dir, "*"))
    file_list = glob.glob("train_data/test/*/*")
    for i in range(total):
        img_path = random.choice(file_list)
        img_cat = os.path.split(os.path.dirname(img_path))[1]
        total_time, infer_time, pred_label, frame = model.predict(img_path)

        if i > 1:
            total_infer_spent_time += infer_time
            total_spent_time += total_time

        if img_cat == pred_label:
            match = match + 1

        print(img_path, str(int(total_time*1000.0)) + 'msec', str(int(infer_time*1000.0)) + 'msec', pred_label) #ここ追加

        tmp_se = pd.Series( [img_cat, pred_label, str(int(total_time * 1000)), str(int(infer_time * 1000)) ], index=list_df.columns )
        list_df = list_df.append( tmp_se, ignore_index=True ) 

    print()
    print('全' + str(total) + '枚 完了！')
    print()
    print("平均処理時間: " + str(int((total_spent_time / (total-1))*1000.0)) + " ms/枚")
    print("平均推論時間: " + str(int((total_infer_spent_time / (total-1))*1000.0)) + " ms/枚")
    print("正解率: " + str(match / total * 100.0))
    return int((total_spent_time / (total-1))*1000.0), int((total_infer_spent_time / (total-1))*1000.0)



In [ ]:
tf_total_time, tf_infer_time = run_inference('saved_model.xml', model_type='openvino', total=50)

更に、IRをOpenVINOの推論エンジン（IE）上で実行してみます。TensorFlowの時と同じ推論スクリプトを使用します。モデルはFP32のままですが、IRに変換することでモデルの内部構造がCPUに最適化され、大きく性能が向上したことが確認できるかと思います。